# selenium on mendix

in case you want to perform identical navigation actions and would like to measure how long it takes to reveice pages, selenium can help.

In mendix Mendix Studio (Pro) or you name widgets. Widgets names show up in class attribute.
For example if widget is named `mx-name-actionButton7` in side a snippet and this snippet is used in a list view several times, selenium `find_elements_by_class_name` can be used to get a list of all elements having a certain name.
```python
lElem=browser.find_elements_by_class_name("mx-name-actionButton7")
```
After that elements `text` attribute can be used to get the one item you want to navigate to.

This notebook navigates to
+ a tab identified by CSS selector `.mx-name-Courses`
+ wait for a `mx-name-actionButton7` named `Quick intro`
+ navigate to it
+ wait for a `mx-name-actionButton7` named `Back`
+ navigate to it
+ wait for `.mx-name-Courses` completing the loop

manually you can repeat the loop as many times you like be setting Jupyter Notebooks run block.

When you are happy with amount of measurements take, results are written via pandas to an Excel file `xlsx` to sheet `rsp` (response). Aggregated values are available in sheet `agg` (min, mean, max). To make sure results are not overwritten by accident, current timestamp (UTC) is part of filename.

Please download selenium drivers and place them in your path before running notebook, an excellent place to start is here on [github](https://github.com/SeleniumHQ/selenium).

In [1]:
import time
import pandas
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.expected_conditions import presence_of_element_located

define constants

In [2]:
sDrvChrome='c:/apps/seleniumDrv/chromedriver.exe'

In [3]:
sUrlBase='http://localhost:8080/'

In [4]:
sUrlLogin=sUrlBase+'login.html'

In [5]:
sUrlHome=sUrlBase+'link/home'

In [6]:
sUrlLearn=sUrlBase+'link/learn'

define functions

In [7]:
def measureGet(browser,sUrl):
    zBeg=time.time()
    browser.get(sUrl)
    zEnd=time.time()
    fResponse=zEnd-zBeg
    print(fResponse)
    return fResponse,zBeg,zEnd

In [8]:
def measureDone(zBeg,sClsName=None):
    zUsr=time.time()
    fResponseUsr=zUsr-zBeg
    if sClsName is not None:
        lElem=browser.find_elements_by_class_name(sClsName)
        showElemLst(lElem,None)
    else:
        lElem=None
    print(fResponseUsr)
    return lElem,fResponseUsr,zBeg,zUsr

In [9]:
class ChkPresent(object):
    def __init__(self,sClsName,sMatch):
        self.sClsName=sClsName
        self.sMatch=sMatch
    def __call__(self,browser):
        lElem=browser.find_elements_by_class_name(self.sClsName)
        try:
            if self.sMatch is None:
                return lElem
            for elem in lElem:
                if elem.text.startswith(self.sMatch):
                    return elem
        except:
            #print('execption')
            #print(repr(lElem))
            pass
        return False

In [10]:
def showElemLst(lElem,sMatch):
    try:
        if sMatch is None:
            for elem in lElem:
                print(elem.text)
        else:
            #print(lElem)
            print(lElem.text)
    except:
        pass


In [11]:
def measureWait(wait,oWeb,sCssSelector,sMatch):
    if oWeb is not None:
        #oWeb.move_to_element(oWeb)
        oWeb.click()
    zBeg=time.time()
    lElem=wait.until(ChkPresent(sCssSelector,sMatch))
    showElemLst(lElem,sMatch)
    zEnd=time.time()
    fResponse=zEnd-zBeg
    print(fResponse)
    return lElem,fResponse,zBeg,zEnd    

initialize

In [12]:
lRsp=[]

In [13]:
browser = webdriver.Chrome(sDrvChrome)

In [14]:
wait = WebDriverWait(browser, 60)

In [15]:
measureGet(browser,sUrlLogin)

2.6494290828704834


(2.6494290828704834, 1625398119.9296508, 1625398122.5790799)

**manual actions**

+ enter credentials
+ switch to role citizen, select `demo_citizen`
+ resize browser window
+ close popups, if needed, home page should show


In [16]:
measureGet(browser,sUrlLearn)

0.47274184226989746


(0.47274184226989746, 1625398196.7114632, 1625398197.184205)

click Course

In [53]:
oCourses=browser.find_element_by_css_selector('.mx-name-Courses')
oTut,fRes,fBeg,fEnd=measureWait(wait,oCourses,"mx-name-actionButton7","Quick intro")
lRsp.append(['course',fRes])

Quick intro
0.6055600643157959


click 3rd tutorial

In [54]:
oBack,fRes,fBeg,fEnd=measureWait(wait,oTut,"mx-name-actionButton9","Back")
lRsp.append(['tutorial',fRes])

Back to overview
1.9986677169799805


click back

In [55]:
oCourse,fRes,fBeg,fEnd=measureWait(wait,oBack,"mx-name-Courses",None)
lRsp.append(['back',fRes])

Courses
0.5609006881713867


show measurements

In [56]:
for tRsp in lRsp:
    s='%-20s %4.1f'%tuple(tRsp)
    print(s)

write result

In [57]:
sResFN='loc01_res_%s.xlsx'%(time.strftime('%Y%m%d_%H%M',time.gmtime()))
oExcel=pandas.ExcelWriter(sResFN)

In [58]:
oRes=pandas.DataFrame(lRsp,columns=['action','duration'])
oRes.to_excel(oExcel,sheet_name='rsp')

In [59]:
oGrp=oRes.groupby('action').agg(['min','mean','max'])
oGrp.to_excel(oExcel,sheet_name='agg')

In [60]:
oExcel.close()